# Лабораторная работа №2. Деревья решений

Выполнили: Рутин Василий, Гордеев Станислав

Все лабораторные работы доступны по адресу: https://github.com/Omenstudio/ML_IFMO_LABS

## 1. Постановка задачи

1. Прочитать теоретическую часть по деревьям решений
2. Описать структуру исходных данных для своего набора:<br>
	a. общие характеристики массива данных: предметная область, количество записей<br>
	b. входные параметры: названия и типы<br>
	c. выходной класс: название и значения<br>
3. Провести серию экспериментов с построением и тестированием деревьев решений (используя DecisionTreeClassifier и RandomForestClassifier), переразбивая исходное множество данных.
4. Осуществить классификацию
5. Сформулировать вывод по использованию деревьев решений для исходной задачи

## 2. Исходные данные

Задача классификации: определение наличия заболевания (диабет) у женщин.<br>
Датасет доступен по адресу: https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

Количество записей: 768<br>
Количество атрибутов: 8 плюс класс.<br>
Все атрибуты представлены числовыми значениями:
1. Количество беременностей (шт.)
2. Концентрация плазмы глюкозы (ммоль/л)
3. Диастолическое давление (мм. рт. столба)
4. Толщина кожной складки (мм.)
5. Содержание инсулина (мкЕд/мл)
6. Индекс массы тела (кг/м2)
7. Предрасположенность по родословной линии (Diabetes pedigree function)
8. Возраст (лет)
9. Класс: наличие диабета (0, 1)

## 3. Классификация и тестирование

Для сравнения различных алгоритмов напишем функцию, которая будет загружать выборку и разбивать её на обучающую и тестовую. 

In [1]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


def load_dataset(split_ratio=None, normalize=True):
    dataset_full = np.loadtxt(open("../pima-indians-diabetes.data.csv", "r"), delimiter=",", skiprows=0, dtype=np.float64)
    inputs = dataset_full[:, :-1]
    outputs = dataset_full[:, -1]
    outputs = outputs.astype(np.int64, copy=False)
    if split_ratio is None:
        return inputs, outputs, None, None
    inputs_train, inputs_test, outputs_train, outputs_test \
        = train_test_split(inputs, outputs, test_size=split_ratio, random_state=random.randint(0, 1000))
    if normalize:
        std_scale = preprocessing.StandardScaler().fit(inputs_train)
        inputs_train = std_scale.transform(inputs_train)
        inputs_test = std_scale.transform(inputs_test)
    return inputs_train, outputs_train, inputs_test, outputs_test

Далее нам потребуется функция, которая будет разбивать выборку на 2 части в определенном соотношении, обучать дерево решений по обучающей выборке, предсказывать по оценивающей выборке и возвращать точность предсказаний.

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import tree


def calc_state(split_ratio):
    res = [split_ratio]
    inputs_train, outputs_train, inputs_test, outputs_test = load_dataset(split_ratio=split_ratio, normalize=True)
    classificators = [
        tree.DecisionTreeClassifier(),
        RandomForestClassifier()
    ]
    for clf in classificators:
        clf.fit(inputs_train, outputs_train)
        res.append(clf.score(inputs_test, outputs_test))
    return res

И наконец сама функция, которая будет перебирать различные "соотношения" и вычислять точность:

In [3]:
def main():
    split_ratio = .5
    print('ratio', '\tDecisionTree', '\tRandomForest')
    while split_ratio < .9:
        ans = calc_state(split_ratio)
        print('{:.1f}\t{:.5f}\t\t{:.5f}'.format(ans[0], ans[1], ans[2]))
        split_ratio += .1

Запускаем и получаем следующие результаты:

In [4]:
main()

ratio 	DecisionTree 	RandomForest
0.5	0.67969		0.71615
0.6	0.67896		0.72885
0.7	0.69145		0.73606
0.8	0.71382		0.72520
0.9	0.66329		0.73121


## 4. Заключение

В ходе лабораторной работы были проведеный эксперименты с классификаторами DecisionTree и RandomForest. 

- "Лес" показывает бОльшую точность, чем "дерево" на всём множестве тестов.
- RandomForest показывает точность сопоставимую с GaussianNaiveBayes алгоритмом.
- RandomForest превосходит по точности метод К соседей.